In [31]:
# Libraries
import os
import numpy as np
!conda install --yes pandas 
import pandas as pd

import cv2
!conda install --yes seaborn 
import seaborn as sns
#!conda install --yes patool 
#import patoolibdataloader
#patoolib.extract_archive('models.zip')

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
import gc
import time
import copy
from tqdm import tqdm
from sklearn.utils import compute_class_weight
from PIL import Image
import math
from glob import glob


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [32]:
# Hyper parameters
#NN='Resnet' ##Choise the Neural Network CNN or Resnet 
NUM_CLASSES = 2 # Labels: 0 - Non IDC, 1 - IDC

#CNN
NUM_EPOCHS_CNN = 1
BATCH_SIZE_CNN  = 128
learning_rate_CNN = 0.002

#Resnet
NUM_EPOCHS_RESNET = 1
BATCH_SIZE_RESNET  = 128
learning_rate_RESNET  = 0.002
run_training = True
retrain = False
find_learning_rate = True

In [33]:
# Device configuration
#torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

'Quadro M4000'

In [34]:
x=[] # x coordinate of an image
SelectedImageData = []
y=[] # y coordinate of an image
ID=[] # ID of a patient
cl=[] # class (0 for non IDC, 1 for IDC)
target=[] # class
matrix = []
i = 0
imagePatches = glob('../IDC_regular_ps50_idx5/**/*.png', recursive=True)
print("ImagePatches length (Total number of images): ", len(imagePatches))
print("imagePatches[0] (Checking if we read the file name): ", imagePatches[0])

# Go over all the images and create a dataset csv file including: x coordinate, y coordinate, target(label: 0-non IDC, 1-IDC), ID - Patient ID
for img in imagePatches:
    # For each image path extract the data from the image name
    imagePatches1=imagePatches[i].split("/")
    imagePatches2=imagePatches1[-1].split("_")
    x=imagePatches2[2].split("x")
    x=x[1]
    y=imagePatches2[3].split("y")
    y=y[1]
    target=imagePatches2[4].split(".png")
    cl=target[0]
   
    if cl=='class0':
        cl=0
    elif cl=='class1':
         cl=1
    ID=imagePatches2[0]
    filename=imagePatches[i]
    SelectedImageData = [ID,x,y,filename,cl]
    column_names = ["patient_id","x","y","filename","target"]
    matrix.append(SelectedImageData)
    i=i+1

ImagePatches length (Total number of images):  277524
imagePatches[0] (Checking if we read the file name):  ../IDC_regular_ps50_idx5/12910/1/12910_idx5_x651_y201_class1.png


In [35]:
base_path = "../IDC_regular_ps50_idx5/"
folder = os.listdir(base_path)

# Add fully defined column into the row
images_df = pd.DataFrame(matrix, columns = column_names)

In [36]:
print("Create csv file")
# Create the dataset csv
images_df.to_csv(r'dataset.csv', index = False) 

Create csv file


In [37]:
files = os.listdir("../IDC_regular_ps50_idx5/")
print("Number of folders (Patients): ", len(files))

Number of folders (Patients):  279


In [38]:
# read csv file and return the created dataset (cancer_perc - precent of cancer images per patinet)
dataset = pd.read_csv("dataset.csv", usecols=['patient_id', 'filename','target'])
cancer_perc = dataset.groupby("patient_id").target.value_counts()/ dataset.groupby("patient_id").target.size()
cancer_perc = cancer_perc.unstack()
print("cancer_perc:" ,cancer_perc)

cancer_perc: target             0         1
patient_id                    
8863        0.788560  0.211440
8864        0.710503  0.289497
8865        0.922753  0.077247
8867        0.901340  0.098660
8913        0.914136  0.085864
...              ...       ...
16568       0.658213  0.341787
16569       0.896142  0.103858
16570       0.408942  0.591058
16895       0.761589  0.238411
16896       0.902396  0.097604

[279 rows x 2 columns]


In [39]:
#Splitting data into train and test: 0.2 for test 0.8 for train
train, test = train_test_split(images_df, stratify=images_df.target, test_size=0.2,train_size=0.8)
print("Train length: %d Test length %d" %(len(train), len(test)))

Train length: 222019 Test length 55505


In [40]:
print(train.head())
#print(test)

       patient_id     x     y  \
245185      14192   901  2351   
27308        9083   701   901   
80151       16166   851  1051   
69839       10269  2201  1201   
107456       9347  1801   951   

                                                 filename  target  
245185  ../IDC_regular_ps50_idx5/14192/0/14192_idx5_x9...       0  
27308   ../IDC_regular_ps50_idx5/9083/0/9083_idx5_x701...       0  
80151   ../IDC_regular_ps50_idx5/16166/0/16166_idx5_x8...       0  
69839   ../IDC_regular_ps50_idx5/10269/0/10269_idx5_x2...       0  
107456  ../IDC_regular_ps50_idx5/9347/0/9347_idx5_x180...       0  


In [46]:
train_cnn = train[["filename","target"]]
test_cnn = test[["filename","target"]]

In [47]:
print(train_cnn.head())
#print(test_cnn)

                                                 filename  target
245185  ../IDC_regular_ps50_idx5/14192/0/14192_idx5_x9...       0
27308   ../IDC_regular_ps50_idx5/9083/0/9083_idx5_x701...       0
80151   ../IDC_regular_ps50_idx5/16166/0/16166_idx5_x8...       0
69839   ../IDC_regular_ps50_idx5/10269/0/10269_idx5_x2...       0
107456  ../IDC_regular_ps50_idx5/9347/0/9347_idx5_x180...       0


In [48]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [49]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_test = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

In [50]:
dataset_train = MyDataset(df_data = train_cnn, transform=trans_train)
dataset_test = MyDataset(df_data = test_cnn,transform=trans_test)

In [51]:
#loader_train = DataLoader(dataset = dataset_train, batch_size=BATCH_SIZE_CNN, shuffle=True, num_workers=0)
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE_CNN, shuffle=True, drop_last=True)

loader_test = DataLoader(dataset = dataset_test, batch_size=BATCH_SIZE_CNN//2, shuffle=False, num_workers=0)
print("Interaction for train and test:")
print(len(loader_train), len(loader_test))

Interaction for train and test:
1734 868


In [52]:
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        #self.conv11 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=2)
        #self.conv12 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)            
        self.bn1 = nn.BatchNorm2d(32)
        #self.bn11 = nn.BatchNorm2d(16)
        #self.bn12 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
        
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        #x = self.pool(F.leaky_relu(self.bn11(self.conv11(x))))
        #x = self.pool(F.leaky_relu(self.bn12(self.conv12(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))   
        x = self.avg(x)       
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [70]:
# Train the model
def train_model(NUM_EPOCHS,learning_rate,loader_train,optimizer,criterion,model):
    print("Train started:")
    model.to(device)
    #model_Cnn.to(device)
    #criterion = nn.CrossEntropyLoss()
    # Loss and optimizer
    #optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)
    loss_array=[]
    x=[]
    Epoch_time=0
    Int100_time=0
    Int100_time1=np.zeros(1735) 
    total_step = len(loader_train)
    test_loss , test_accuracy = [], []
    for epoch in range(NUM_EPOCHS):
        #tqdm(range(2), desc = 'epoch')
        torch.cuda.synchronize()
        start= time.time()
        tk0 = tqdm(loader_train, total=int(len(loader_train)))
        
        for i, (filename, target) in enumerate(tk0):
            #tqdm(range(len(loader_train)), desc = 'epoch')
            #tq(range(len(loader_train)))
            #print(filename)
            #print(target)
            #break
            filename = filename.to(device)
            target = target.to(device)
        
        # Forward pass
            #torch.cuda.empty_cache()        
            outputs = model(filename)
            #gc.collect()
        
        #print(outputs)
            loss = criterion(outputs, target)
        
        
        # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
       
            if (i+1) % 100 == 0:
                torch.cuda.synchronize()
                stop= time.time()
                Int100_time=stop-start
                Int100_time1[i]=Int100_time1[i]+Int100_time           
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, time:{} min' 
                           .format(epoch+1, NUM_EPOCHS, i+1, total_step, loss.item(),Int100_time/60))
                loss_array.append(loss.item())
            elif (i+1) % 1735 == 0:
                torch.cuda.synchronize()
                stop= time.time()            
                Epoch_time=Epoch_time+(stop-start)/60
                
                # Save the model checkpoint
                torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss_array': loss_array,
                }, 'models/model.pth')
                  
                #torch.save(model.state_dict(), 'models/model_res.ckpt')
                """
                if (epoch+1) % NUM_EPOCHS == 0:
                                      
                    Test_acc =Test_model(model)
                   
                    print ('All_Epochs [{}],Loss: {:.4f},Test_accuracy:{} %,Epoch_total_time:{} min' 
                   .format(NUM_EPOCHS,loss.item(),Test_acc,Epoch_time))
                    # Save the model checkpoint
                    torch.save(model.state_dict(), 'models/model_res.ckpt')
                    
                else:
                        torch.cuda.synchronize()
                        print ('Epoch [{}],Loss: {:.4f},Epoch_time:{} min' 
                                 .format(epoch+1,loss.item(),Epoch_time))
                        loss_array.append(loss.item())

    # save model checkpoint
    torch.save({
            'epoch': NUM_EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
            'Accurcy':Test_acc
            }, '../DeepLearningProjectNew/models/model.pth')
   """
                
    return loss_array

In [71]:
# Test the model
def Test_model(model_Cnn):
    model_Cnn.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    confusion_matrix = torch.zeros(2, 2)
    with torch.no_grad():
        correct = 0
        total = 0
        for filename, target in loader_test:
            filename = filename.to(device)
            target = target.to(device)
            outputs = model_Cnn(filename)               
            temp, predicted = torch.max(outputs.data, 1)               
            total += target.size(0)
            correct += (predicted == target).sum().item()
            for t, p in zip(target.view(-1), predicted.view(-1)):
                  confusion_matrix[t.long(), p.long()] += 1                    
        Test_acc =format(100 * correct / total)
    #print(confusion_matrix)    
    return Test_acc, confusion_matrix   

In [72]:
model_Cnn = SimpleCNN()

In [73]:
criterion_cnn = nn.CrossEntropyLoss()
# Loss and optimizer
optimizer_cnn = torch.optim.Adamax(model_Cnn.parameters(), lr=learning_rate_CNN)

In [ ]:
loss_array = train_model(NUM_EPOCHS_CNN,learning_rate_CNN,loader_train, optimizer_cnn,criterion_cnn,model_Cnn)

Train started:


  6%|▌         | 100/1734 [01:39<32:29,  1.19s/it]

Epoch [1/1], Step [100/1734], Loss: 0.3395, time:1.656271266937256 min


 10%|█         | 175/1734 [02:56<29:46,  1.15s/it]

In [ ]:
Test_acc, confusion_matrix = Test_model(model)
#print(Test_acc)
print("confusion_matrix: ", confusion_matrix)

print ('All_Epochs [{}],Loss: {:.4f},Test_accuracy:{} %,Epoch_total_time:{} min' 
.format(NUM_EPOCHS,loss.item(),Test_acc,Epoch_time))

In [26]:
# accuracy plots
plt.figure(figsize = (10, 7))
plt.plot(Test_acc, color = 'green', label = 'test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('../DeepLearningProjectNew/models/test accuracy.png')
plt.show()

# loss plots
plt.figure(figsize=(10, 7))
plt.plot(loss_array, color='orange', label='train loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('../DeepLearningProjectNew/models/Train_Loss.png')
plt.show()  

NameError: name 'Test_acc' is not defined

<Figure size 720x504 with 0 Axes>

End of CNN Module Start of Resnet Module

'Quadro M4000'

In [168]:
def weights(device,train_df):    
    weights = compute_class_weight(y=train_df.target.values, class_weight = "balanced", classes = train_df.target.unique())    
    class_weights = torch.FloatTensor(weights)
    if device.type == "cuda":
        class_weights = class_weights.cuda()
    print(class_weights)
    train_df.target.unique()
    return class_weights

In [169]:
model_resnet = torchvision.models.resnet18(pretrained=False)
#if run_training:
#    model.load_state_dict(torch.load("../input/pretrained-pytorch-models/resnet18-5c106cde.pth"))
num_features = model_resnet.fc.in_features
print(num_features)

model_resnet.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    
    nn.Linear(256, NUM_CLASSES))

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model_resnet.apply(init_weights)
model_resnet = model_resnet.to(device)

512


Number of features:2048


In [170]:
class_weights = weights(device,train_cnn)
print(class_weights)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adamax(model_resnet.fc.parameters()) #, end_lr)#torch.optim.SGD(model.fc.parameters(), start_lr)

tensor([0.6982, 1.7612], device='cuda:0')
tensor([0.6982, 1.7612], device='cuda:0')


In [172]:
loss_array_RESNET = train_model_resnet(NUM_EPOCHS_RESNET,learning_rate_RESNET,loader_train,optimizer,criterion,model_resnet)

Resnet_Train started:
Epoch [1/1], Step [100/1734], Loss: 0.7231,Int100_time:2.7729415853818256 min
Epoch [1/1], Step [200/1734], Loss: 0.4986,Int100_time:5.237090249856313 min
Epoch [1/1], Step [300/1734], Loss: 0.4683,Int100_time:7.7035147507985435 min
Epoch [1/1], Step [400/1734], Loss: 0.5193,Int100_time:9.975064321358998 min
Epoch [1/1], Step [500/1734], Loss: 0.4314,Int100_time:12.302738801638286 min
Epoch [1/1], Step [600/1734], Loss: 0.4673,Int100_time:14.531976584593455 min
Epoch [1/1], Step [700/1734], Loss: 0.4648,Int100_time:16.736066663265227 min
Epoch [1/1], Step [800/1734], Loss: 0.4282,Int100_time:19.038437879085542 min
Epoch [1/1], Step [900/1734], Loss: 0.5902,Int100_time:22.094458039601644 min
Epoch [1/1], Step [1000/1734], Loss: 0.4160,Int100_time:25.2004869778951 min
Epoch [1/1], Step [1100/1734], Loss: 0.4629,Int100_time:28.219782265027366 min
Epoch [1/1], Step [1200/1734], Loss: 0.5316,Int100_time:31.340622301896413 min
Epoch [1/1], Step [1300/1734], Loss: 0.3821

In [174]:
Test_acc, confusion_matrix = Test_model(model_resnet)

In [175]:
print(Test_acc)
print(confusion_matrix)

76.15530132420503


In [176]:
def model50(NUM_CLASSES):
    model = torchvision.models.resnet50(pretrained=False)
    #model.load_state_dict(torch.load("input/breastcancermodel/resnet50-19c8e357.pth"))
    num_features = model.fc.in_features
    print("Number of features:"+str(num_features))
    model.fc = nn.Sequential(
               nn.Linear(num_features, 512),
               nn.ReLU(),
               nn.BatchNorm1d(512),
               nn.Dropout(0.25),
               nn.Linear(512, 256),
               nn.ReLU(),
               nn.BatchNorm1d(256),
               nn.Dropout(0.25),
               nn.Linear(256, 128),
               nn.ReLU(),
               nn.BatchNorm1d(128),
               nn.Dropout(0.25),
               nn.Linear(128, 64),
               nn.ReLU(),
               nn.BatchNorm1d(64),
               nn.Dropout(0.25),
               nn.Linear(64, 32),
               nn.ReLU(),
               nn.BatchNorm1d(32),
               nn.Dropout(0.25),
               nn.Linear(32, 16),
               nn.ReLU(),
               nn.BatchNorm1d(16),
               nn.Dropout(0.25),
               nn.Linear(16, 8),
               nn.ReLU(),
               nn.BatchNorm1d(8),
               nn.Dropout(0.25),
               nn.Linear(8, NUM_CLASSES))

    def init_weights(m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    model.apply(init_weights)
    model = model.to(device)
    return model

In [181]:
torch.cuda.empty_cache() 

In [182]:
# Create resnet model
model_resnet2 = model50(NUM_CLASSES)

Number of features:2048


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 7.94 GiB total capacity; 7.14 GiB already allocated; 192.00 KiB free; 7.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [178]:
loss_array_RESNET2 = train_model_resnet(NUM_EPOCHS_RESNET,learning_rate_RESNET,loader_train,optimizer,criterion,model_resnet2)

Resnet_Train started:


RuntimeError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 7.94 GiB total capacity; 7.04 GiB already allocated; 30.19 MiB free; 7.13 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF